## Dev env configuration

In [1]:
# install the required python packages
# !pip install numpy pandas yfinance matplotlib mplfinance seaborn

## Imports

In [2]:
# import the necessary packages
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
import yfinance as yf
import pandas as pd

## Configuring our date range and ticker

In [27]:
# set the start and end dates for our market data request to be TTM
end_date = datetime(year=2025, month=3, day=1)
start_date = end_date - timedelta(days=365)

# set the name of the ticker we want to download market data for
ticker = "AAPL"

## Computing daily 50 and 200 moving averages

In [37]:
# download daily market data
df_daily = yf.download(
    tickers=ticker,
    start=start_date,
    end=end_date,
    interval="1d",
    group_by="ticker",
    auto_adjust=True,
    progress=False
)
df_daily

Ticker            AAPL                                              
Price             Open        High         Low       Close    Volume
Date                                                                
2024-03-01  178.706175  179.681565  176.546375  178.815659  73488000
2024-03-04  175.322153  176.068628  172.973243  174.277100  81510100
2024-03-05  169.957472  171.231455  168.822830  169.320480  95132400
2024-03-06  170.256065  170.435227  167.887245  168.325180  68587700
2024-03-07  168.355038  169.927615  167.698152  168.205750  71765100
...                ...         ...         ...         ...       ...
2025-02-24  244.929993  248.860001  244.419998  247.100006  51326400
2025-02-25  248.000000  250.000000  244.910004  247.039993  48013300
2025-02-26  244.330002  244.979996  239.130005  240.360001  44433600
2025-02-27  239.410004  242.460007  237.059998  237.300003  41153600
2025-02-28  236.949997  242.089996  230.199997  241.839996  56796200

[250 rows x 5 columns]

In [41]:
df_daily.loc[:, (ticker, "50MA")] = df_daily[ticker, "Close"].transform(lambda x: x.rolling(window=50).mean())
df_daily

Ticker            AAPL                                                \
Price             Open        High         Low       Close    Volume   
Date                                                                   
2024-03-01  178.706175  179.681565  176.546375  178.815659  73488000   
2024-03-04  175.322153  176.068628  172.973243  174.277100  81510100   
2024-03-05  169.957472  171.231455  168.822830  169.320480  95132400   
2024-03-06  170.256065  170.435227  167.887245  168.325180  68587700   
2024-03-07  168.355038  169.927615  167.698152  168.205750  71765100   
...                ...         ...         ...         ...       ...   
2025-02-24  244.929993  248.860001  244.419998  247.100006  51326400   
2025-02-25  248.000000  250.000000  244.910004  247.039993  48013300   
2025-02-26  244.330002  244.979996  239.130005  240.360001  44433600   
2025-02-27  239.410004  242.460007  237.059998  237.300003  41153600   
2025-02-28  236.949997  242.089996  230.199997  241.839996  56796200   

Ticker                  
Price             50MA  
Date                    
2024-03-01         NaN  
2024-03-04         NaN  
2024-03-05         NaN  
2024-03-06         NaN  
2024-03-07         NaN  
...                ...  
2025-02-24  240.485810  
2025-02-25  240.476652  
2025-02-26  240.359467  
2025-02-27  240.151713  
2025-02-28  240.031363  

[250 rows x 6 columns]

In [31]:
# calculate the daily 50 and 200 MAs
df_daily[:, "50MA"] = df_daily["Close"].groupby("Ticker").transform(lambda x: x.rolling(window=50).mean())
df_daily

KeyError: 'Close'

In [36]:
df_daily["AAPL"]

Price,Open,High,Low,Close,Volume
Date,,,,,
2024-03-01,178.706175,179.681565,176.546375,178.815659,73488000
2024-03-04,175.322153,176.068628,172.973243,174.277100,81510100
2024-03-05,169.957472,171.231455,168.822830,169.320480,95132400
2024-03-06,170.256065,170.435227,167.887245,168.325180,68587700
2024-03-07,168.355038,169.927615,167.698152,168.205750,71765100
...,...,...,...,...,...
2025-02-24,244.929993,248.860001,244.419998,247.100006,51326400
2025-02-25,248.000000,250.000000,244.910004,247.039993,48013300
2025-02-26,244.330002,244.979996,239.130005,240.360001,44433600


In [17]:
# # construct additional plots for the MAs
# addt_plots = [
#     mpf.make_addplot(df_daily["50MA"], color="blue", width=1.5, label="50MA"),
#     mpf.make_addplot(df_daily["200MA"], color="red", width=1.5, label="200MA"),
# ]

# # plot the daily OHLCV and MAs
# mpf.plot(
#     df_daily
    
# )

## Computing weekly 10 and 40 moving averages

## Compututing monthly 10 moving average